Two functions were written oneis used to unpickle all the batches and the second is used to convert the unpickled data back into an image-like array.

In [1]:
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

In [2]:
batch_1 = unpickle('cifar-10-batches-py/data_batch_1')
batch_2 = unpickle('cifar-10-batches-py/data_batch_2')
batch_3 = unpickle('cifar-10-batches-py/data_batch_3')
batch_4 = unpickle('cifar-10-batches-py/data_batch_4')
batch_5 = unpickle('cifar-10-batches-py/data_batch_5')
batch_test = unpickle('cifar-10-batches-py/test_batch')

In [3]:
def img_process(arr):
    im_list = []
    for img in arr:
        im_list.append(img.reshape((32,32,3)))
    return np.array(im_list)

In [5]:
import numpy as np

processed_1 = img_process(batch_1[b'data'])
processed_2 = img_process(batch_2[b'data'])
processed_3 = img_process(batch_3[b'data'])
processed_4 = img_process(batch_4[b'data'])
processed_5 = img_process(batch_5[b'data'])
processed_test = img_process(batch_test[b'data'])

(32, 32, 3)

Batches are combined into one large training set and a sepereate testing set from the sixth batch.

In [23]:
import numpy as np
from keras.utils import np_utils

train_tensors = np.concatenate((processed_1, processed_2, processed_3, processed_4, processed_5))
target_labels = np.concatenate((batch_1[b'labels'], batch_2[b'labels'], batch_3[b'labels'], batch_4[b'labels'], batch_5[b'labels']))
train_targets = np_utils.to_categorical(target_labels, 10)

test_tensors = img_process(batch_test[b'data'])
test_targets = np_utils.to_categorical(batch_test[b'labels'], 10)

Model is created with the ResNet inspired architecture. 

In [18]:
from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout, MaxPooling2D, Conv2D, GlobalAveragePooling2D


model = Sequential()

model.add(Conv2D(filters=16, kernel_size=3, padding='same', activation='relu', input_shape=train_tensors[0].shape))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(filters=32, kernel_size=3, padding='same', activation='relu'))
model.add(Conv2D(filters=32, kernel_size=3, padding='same', activation='relu'))
model.add(Conv2D(filters=32, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(filters=64, kernel_size=3, padding='same', activation='relu'))
model.add(Conv2D(filters=64, kernel_size=3, padding='same', activation='relu'))
model.add(Conv2D(filters=64, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(filters=128, kernel_size=3, padding='same', activation='relu'))
model.add(Conv2D(filters=128, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(10, activation='softmax'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_10 (Conv2D)           (None, 32, 32, 16)        448       
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 16, 16, 16)        0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 16, 16, 32)        4640      
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 16, 16, 32)        9248      
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 16, 16, 32)        9248      
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 8, 8, 32)          0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 8, 8, 64)          18496     
__________

In [19]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

Model it trained on the cifar-10 dataset

In [45]:
from keras.callbacks import ModelCheckpoint

epochs = 20


checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.cifar_7.hdf5', 
                               verbose=1, save_best_only=True)

model.fit(train_tensors, train_targets, 
          validation_data=(test_tensors, test_targets),
          epochs=epochs, batch_size=31, callbacks=[checkpointer], verbose=1)

Train on 50000 samples, validate on 10000 samples
Epoch 1/20
50000/50000 [==============================] - 210s 4ms/step - loss: 1.5111 - acc: 0.4905 - val_loss: 1.3349 - val_acc: 0.5312

Epoch 00001: val_loss improved from inf to 1.33486, saving model to saved_models/weights.best.cifar_7.hdf5
Epoch 2/20
50000/50000 [==============================] - 203s 4ms/step - loss: 1.5215 - acc: 0.4876 - val_loss: 1.7847 - val_acc: 0.3504

Epoch 00002: val_loss did not improve from 1.33486
Epoch 3/20
50000/50000 [==============================] - 248s 5ms/step - loss: 1.5390 - acc: 0.4797 - val_loss: 1.4084 - val_acc: 0.5167

Epoch 00003: val_loss did not improve from 1.33486
Epoch 4/20
50000/50000 [==============================] - 458s 9ms/step - loss: 1.5745 - acc: 0.4716 - val_loss: 1.4875 - val_acc: 0.4619

Epoch 00004: val_loss did not improve from 1.33486
Epoch 5/20
50000/50000 [==============================] - 696s 14ms/step - loss: 1.5725 - acc: 0.4672 - val_loss: 1.4778 - val_acc: 0.